# Generating features from images with a deep network

The goal of this notebook is to generate (meaningful) feature vectors from raw images. These features vectors will be easier to handle than raw images with the methods seen in the course.
To generate these features, we will use a pre-trained Convolutional Neural Network (CNN). We will not train it, we will just use it as a feature generator. Since it has already been trained on images, it should be able to extract meaningful features from them, that should help for other different image analysis tasks.

The pre-trained CNN available in Tensorflow have been trained on a particular task (classification of images into precise categories) and we want to use them for another task (classification in other categories). The idea is to get rid of the original output layer(s) of the CNN and collect the output of one of the inner layer. This layer should contain meaningful features.

As this is becoming a standard process, Tensorflow has made things simple for loading a pre-trained model and getting its inner layer features. This notebook shows how this can be done.

This notebook is inspired from this [Google Colab example](https://www.tensorflow.org/hub/tutorials/tf2_image_retraining). 

In [ ]:
#  Modules to install
# !pip install tensorflow
# !pip install tensorflow_hub

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras import layers

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

## Load the dataset and make a train and test set.

In [ ]:
# Load the dataset
data_dir = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

In [ ]:
# Load the pre-trained model
module_selection = ("mobilenet_v2_100_224", 224)# Others to try: ("inception_v3", 299), ("resnet_v2_50", 299)
handle_base, pixels = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/imagenet/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels) # input size of the neural network
print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))

In [ ]:
# Creating the training and validation sets
SEED = 123 # This seed will make the random split of images between train and test set fixed
BATCH_SIZE = 32 # group images in batches of BATCH_SIZE

dataflow_kwargs = dict(interpolation="bilinear",  validation_split=.20,
                       seed=SEED, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
# to change labels from integers to one-hot vectors add: label_mode="categorical"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir, subset="training", shuffle=True, **dataflow_kwargs)
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir, subset="validation", shuffle=True, **dataflow_kwargs)

In [ ]:
class_names = train_ds.class_names
print('Data classes:',class_names)
num_classes = len(class_names)

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1): # take a batch
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")
print('Image size:',images[i].shape)

## Create the neural network
To use the pre-trained model, we need to adapt the new images to the format expected by the pre-trained model. The images need to be resizes and normalized (pixel values between 0 and 1).

In [ ]:
# We create a model with the pre-trained part. We add at the end a dense layer (not mandatory for obtaining the features)
print("Building model with", MODULE_HANDLE)
model = tf.keras.Sequential([
    #----- Input formating and preprocessing
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    layers.experimental.preprocessing.Rescaling(1./255),
    #----- the pre-trained model loaded with a single line:
    hub.KerasLayer(MODULE_HANDLE, trainable=False, name='embeddings'),
    #----- some additional layers to perform a classification task afterwards (untrained, to be trained)
    #tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(num_classes,
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])

model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

In order to collect the output of an inner layer we create an intermediate model. We could have omitted the last layer in the previous definition, but to be more general we define a complete model and then show how to extract a part of it.

In [ ]:
# Create the intermediate model that output features
layer_name='embeddings'
layer_output=model.get_layer(layer_name).output
intermediate_model=tf.keras.models.Model(inputs=model.input, outputs=layer_output)
embeddings_size = intermediate_model.output.shape[1]
print('Embeddings vector size (output of the intermediate NN):',embeddings_size)

In [ ]:
# let us collect the embeddings by iterating over batches.
embeddings = None
labels_vector = None
for images, labels in valid_ds:
    embeddings_batch = intermediate_model.predict(images)
    if embeddings is None:
        embeddings = embeddings_batch
        labels_vector = labels.numpy() # convert tensors to numpy arrays
    else:
        embeddings = np.vstack((embeddings, embeddings_batch))
        labels_vector = np.hstack((labels_vector, labels))
print('Embeddings shape:', embeddings.shape)

In [ ]:
plt.imshow(embeddings)
plt.colorbar()
plt.xlabel('Features')
plt.ylabel('Samples')
plt.title('Feature values')
plt.show()

The features are positive because there a going out of a relu function. Also, many of them are zero.

In [ ]:
# Save the dataset and labels
np.savez_compressed('dataPatternRecognition.npz', embeddings=embeddings, labels_vector=labels_vector, class_names=class_names)

# Training the full model (Not required for the course)
This is just a demo of NN training in Tensorflow

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9), 
    #loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])


In [ ]:
hist = model.fit(train_ds, validation_data=valid_ds, epochs=4).history 

In [ ]:
plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
#plt.ylim([0,2])
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])

plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")
#plt.ylim([0,1])
plt.plot(hist["accuracy"])
plt.plot(hist["val_accuracy"])
plt.show()

## Visualizing the results of the classification

In [ ]:
plt.figure(figsize=(20, 15))
for images, labels in valid_ds.take(1):
    predictions = model.predict(images)
    for i in range(30):
        ax = plt.subplot(5, 6, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        pred_i = class_names[np.argmax(predictions[i])]
        color = "green" if pred_i == class_names[labels[i]] else "red"
        plt.title(class_names[labels[i]]+' Classified: '+pred_i, color=color)
        #plt.title(class_names[np.argmax(pred[i])])
        plt.axis("off")